## 数据解析

### 原始数据=>歌单数据 (此系统暂时只使用部分数据)
抽取 _**歌单名称，歌单id，收藏数，所属分类**_ 4个歌单维度的信息 <br>
抽取 _**歌曲id，歌曲名，歌手，歌曲热度**_ 等4个维度信息歌曲的信息<br>

组织成如下格式：
```
漫步西欧小镇上##小语种,旅行##69413685##474	18682332::Wäg vo dir::Joy Amelie::70.0	4335372::Only When I Sleep::The Corrs::60.0	2925502::Si Seulement::Lynnsha::100.0	21014930::Tu N'As Pas Cherché...::La Grande Sophie::100.0	20932638::Du behöver aldrig mer vara rädd::Lasse Lindh::25.0	17100518::Silent Machine::Cat Power::60.0	3308096::Kor pai kon diew : ชอไปคนเดียว::Palmy::5.0	1648250::les choristes::Petits Chanteurs De Saint Marc::100.0	4376212::Paddy's Green Shamrock Shore::The High Kings::25.0	2925400::A Todo Color::Las Escarlatinas::95.0	19711402::Comme Toi::Vox Angeli::75.0	3977526::Stay::Blue Cafe::100.0	2538518::Shake::Elize::85.0	2866799::Mon Ange::Jena Lee::85.0	5191949::Je M'appelle Helene::Hélène Rolles::85.0	20036323::Ich Lieb' Dich Immer Noch So Sehr::Kate & Ben::100.0

### 歌单数据=>推荐系统格式数据
主流的python推荐系统框架，支持的最基本数据格式为movielens dataset，其评分数据格式为 user item rating timestamp

In [66]:
import surprise 
import lightfm

### 1.针对用户推荐 网易云音乐(每日30首歌/7首歌)
### 2.针对歌曲 在你听某首歌的时候，找“相似歌曲”

In [67]:
# 解析成userid itemid rating timestamp行格式
def is_null(s): 
    return len(s.split(","))>2

def parse_song_info(song_info):
    try:
        song_id, name, artist, popularity = song_info.split("::")
        #return ",".join([song_id, name, artist, popularity])
        return ",".join([song_id,"1.0",'1300000'])
    except Exception as e:
        #print (e)
        #print (song_info)
        return ""

def parse_playlist_line(in_line):
    try:
        contents = in_line.strip().split("\t")
        name, tags, playlist_id, subscribed_count = contents[0].split("##")
        songs_info = map(lambda x:playlist_id+","+parse_song_info(x), contents[1:])
        songs_info = filter(is_null, songs_info)
        return "\n".join(songs_info)
    except Exception as e:
        print (e)
        return False


def parse_file(in_file, out_file):
    out = open(out_file, 'w', encoding='utf-8')
    for line in open(in_file, encoding='utf-8'):
        result = parse_playlist_line(line)
        if(result):
            out.write(result.strip()+"\n")
    out.close()

In [68]:
parse_file("./songlist2songs.txt", "./163_music_suprise_format.txt")

### 保存歌单和歌曲信息备用
需要保存 **歌单id=>歌单名** 和 **歌曲id=>歌曲名** 的信息后期备用。

In [69]:
import pickle

def parse_playlist_get_info(in_line, playlist_dic, song_dic):
    contents = in_line.strip().split("\t")
    name, tags, playlist_id, subscribed_count = contents[0].split("##")
    # print(name, tags, playlist_id, subscribed_count)
    playlist_dic[playlist_id] = name
    print("playlist_id, name", playlist_id, name)
    print(contents)
    for song in contents[1:]:
        # print("song: ",song)
        try:
            song_id, song_name, artist, popularity = song.split("::")
            # print("song_id: ", song_id, song_name, artist)
            song_dic[song_id] = song_name+"\t"+artist
        except:
            print ("song format error")
            print (song+"\n")

def parse_song_get_info(in_line, song_dic):
    contents = in_line.strip().split("\t")
    # print(contents)
    try:
        song_id, song_name, artist, popularity = contents[0].split("::")
        song_dic[song_id] = song_name+"\t"+artist
    except:
        print ("song format error")
        print (contents[0]+"\n")

def parse_file_songlist(in_file_songlist, out_playlist, out_song):
    #从歌单id到歌单名称的映射字典
    playlist_dic = {}
    #从歌曲id到歌曲名称的映射字典
    song_dic = {}
    for line in open(in_file_songlist, encoding='utf-8'):
        parse_playlist_get_info(line, playlist_dic, song_dic)
    # for line in open(in_file_song, encoding='utf-8'):
    #     parse_song_get_info(line, song_dic)
    #把映射字典保存在二进制文件中
    pickle.dump(playlist_dic, open(out_playlist,"wb")) 
    pickle.dump(song_dic, open(out_song,"wb"))
    
    
def parse_file2(in_file, out_playlist, out_song):
    #从歌单id到歌单名称的映射字典
    playlist_dic = {}
    #从歌曲id到歌曲名称的映射字典
    song_dic = {}
    for line in open(in_file, encoding='utf-8'):
        parse_playlist_get_info(line, playlist_dic, song_dic)
    #把映射字典保存在二进制文件中
    pickle.dump(playlist_dic, open(out_playlist,"wb")) 
    #可以通过 playlist_dic = pickle.load(open("playlist.pkl","rb"))重新载入
    pickle.dump(song_dic, open(out_song,"wb"))

In [70]:
parse_file_songlist("./songlist2songs.txt", "./pro_data/playlist.pkl", "./pro_data/song.pkl")

playlist_id, name 10009377589 欧美R&B入坑曲：封神旋律 是谁的一代青春
['欧美R&B入坑曲：封神旋律 是谁的一代青春##欧美 R&B/Soul 90后 ##10009377589##78', '2772113::I wanted you::Ina Wroldsen::100', '5100462::Dilemma::Nelly|Kelly Rowland::100', '28718313::The Way I Still Love You::Reynard Silva::100', '518904648::Paris in the Rain::Lauv::100', '1786865::Because Of You::Ne-Yo::100', '1571835::Call You Tonight::Johnta Austin::100', '461544117::Starboy::The Weeknd|Daft Punk::100', '29133008::Before You Break My Heart::Stevie Hoang::100', '479223413::I Like Me Better::Lauv::100', '29412405::In The Shadow Of The Sun::Professor Green::100', '18513043::Love you like I do::Jamillions::100', '431610014::Starboy::The Weeknd|Daft Punk::100', '1830419924::Peaches::Justin Bieber|Daniel Caesar|Giveon::100', '17793578::Be On You::Flo Rida|Ne-Yo::100', "2051548110::Nothin' on Me::Leah Marie Perez::100", '1934649::Shape Of My Heart::Sting::100', '1955978156::Shut up My Moms Calling::Hotel Ugly::100', '1420972635::Come Around Me::Justin Bieber:

## 使用推荐系统库Surprise构建模型

In [71]:
import os
import io

from surprise import KNNBaseline, Reader
from surprise import Dataset

# 重建歌单id到歌单名的映射字典
id_name_dic = pickle.load(open("./pro_data/playlist.pkl","rb"))
print("加载歌单id到歌单名的映射字典完成...")
# 重建歌单名到歌单id的映射字典
name_id_dic = {}
for playlist_id in id_name_dic:
    name_id_dic[id_name_dic[playlist_id]] = playlist_id
print("加载歌单名到歌单id的映射字典完成...")


file_path = os.path.expanduser('./163_music_suprise_format.txt')
# 指定文件格式
reader = Reader(line_format='user item rating timestamp', sep=',')
# 从文件读取数据
music_data = Dataset.load_from_file(file_path, reader=reader)
# 计算歌曲和歌曲之间的相似度
print("构建数据集...")
trainset = music_data.build_full_trainset()
#sim_options = {'name': 'pearson_baseline', 'user_based': False}

加载歌单id到歌单名的映射字典完成...
加载歌单名到歌单id的映射字典完成...
构建数据集...


In [72]:
list(id_name_dic.keys())[2]

'9381079097'

In [73]:
print(id_name_dic[list(id_name_dic.keys())[2]])

华语R&B：百听不厌的华语节奏布鲁斯


In [74]:
trainset.n_items

38551

In [75]:
trainset.n_users

634

### 查找最近的user(在这里是歌单)

In [76]:
print("开始训练模型...")
# sim_options = {'user_based': False}
#algo = KNNBaseline(sim_options=sim_options)
algo = KNNBaseline()
algo.fit(trainset)

current_playlist = list(name_id_dic.keys())[39]
print("歌单名称", current_playlist)

# 取出近邻
# 映射名字到id
playlist_id = name_id_dic[current_playlist]
print("歌单id", playlist_id)
# 取出来对应的内部user id => to_inner_uid
playlist_inner_id = algo.trainset.to_inner_uid(playlist_id)
print("内部id", playlist_inner_id)

playlist_neighbors = algo.get_neighbors(playlist_inner_id, k=10)

# 把歌曲id转成歌曲名字
# to_raw_uid映射回去
playlist_neighbors = (algo.trainset.to_raw_uid(inner_id)
                       for inner_id in playlist_neighbors)
playlist_neighbors = (id_name_dic[playlist_id]
                       for playlist_id in playlist_neighbors)

print()
print("和歌单 《", current_playlist, "》 最接近的10个歌单为：\n")
for playlist in playlist_neighbors:
    print(playlist, algo.trainset.to_inner_uid(name_id_dic[playlist]))

开始训练模型...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
歌单名称 欧美治愈日记本 · 笑着面对痛点
歌单id 10085324532
内部id 42

和歌单 《 欧美治愈日记本 · 笑着面对痛点 》 最接近的10个歌单为：

倾吐隐秘情绪｜百位R&B歌手精选 97
节奏控 | 戴上耳机世界都是你的 139
欧美R&B || 慵懒旋律搭配成熟磁嗓 182
高级感｜氛围感｜舒缓英文歌｜持续更新 190
旋律R&B｜微醺 性感 浪漫氛围感 192
评论过万的英文歌与潜力股 199
【欧美风】霉霉/戳爷/比伯/萌德/断眉 209
“你应该是洒脱的 自由的 勇敢的” 215
微醺欧美：玫瑰沉溺晚霞色彩的海浪 218
节奏控 | 那些超带感的英文歌 248


### 针对用户进行预测

In [77]:
import pickle
# 重建歌曲id到歌曲名的映射字典
song_id_name_dic = pickle.load(open("./pro_data/song.pkl","rb"))
print("加载歌曲id到歌曲名的映射字典完成...")
# 重建歌曲名到歌曲id的映射字典
song_name_id_dic = {}
for song_id in song_id_name_dic:
    song_name_id_dic[song_id_name_dic[song_id]] = song_id
print("加载歌曲名到歌曲id的映射字典完成...")

加载歌曲id到歌曲名的映射字典完成...
加载歌曲名到歌曲id的映射字典完成...


In [78]:
#内部编码的4号用户
user_inner_id = 42
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(user_inner_id, song, r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 42         item: 2732       r_ui = 1.00   est = 1.00   {'was_impossible': False} I Can Do It With a Broken Heart	Taylor Swift
user: 42         item: 2733       r_ui = 1.00   est = 1.00   {'was_impossible': False} Dog Days Are Over	Florence + The Machine
user: 42         item: 2734       r_ui = 1.00   est = 1.00   {'was_impossible': False} Cozy	Jeremy Zucker|Lauv|Alexander 23
user: 42         item: 2735       r_ui = 1.00   est = 1.00   {'was_impossible': False} we can't be friends (wait for your love)	Ariana Grande
user: 42         item: 2736       r_ui = 1.00   est = 1.00   {'was_impossible': False} Take Yourself Home	Troye Sivan
user: 42         item: 2737       r_ui = 1.00   est = 1.00   {'was_impossible': False} thank u, next	Ariana Grande
user: 42         item: 2738       r_ui = 1.00   est = 1.00   {'was_impossible': False} Enough	Jess Glynne
user: 42         item: 2739       r_ui = 1.00   est = 1.00   {'was_impossible': False} always, i'll care	Jeremy Zucker
user: 42        

### 用矩阵分解进行预测

In [79]:
### 使用NMF
from surprise import NMF
from surprise import Dataset

file_path = os.path.expanduser('./163_music_suprise_format.txt')
# 指定文件格式
reader = Reader(line_format='user item rating timestamp', sep=',')
# 从文件读取数据
music_data = Dataset.load_from_file(file_path, reader=reader)
# 构建数据集和建模
algo = NMF()
trainset = music_data.build_full_trainset()
algo.fit(trainset)

In [80]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(algo.trainset.to_raw_uid(user_inner_id), algo.trainset.to_raw_iid(song), r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 9388585797 item: 2128787688 r_ui = 1.00   est = 1.00   {'was_impossible': False} Saturn	SZA
user: 9388585797 item: 2004562490 r_ui = 1.00   est = 1.00   {'was_impossible': False} Kill Bill	SZA
user: 9388585797 item: 2004563430 r_ui = 1.00   est = 1.00   {'was_impossible': False} Snooze	SZA
user: 9388585797 item: 1806905006 r_ui = 1.00   est = 1.00   {'was_impossible': False} Good Days	SZA
user: 9388585797 item: 2005160715 r_ui = 1.00   est = 1.00   {'was_impossible': False} Open Arms	SZA|Travis Scott
user: 9388585797 item: 1899918521 r_ui = 1.00   est = 1.00   {'was_impossible': False} I Hate U	SZA
user: 9388585797 item: 1988683696 r_ui = 1.00   est = 1.00   {'was_impossible': False} Shirt	SZA
user: 9388585797 item: 2004563425 r_ui = 1.00   est = 1.00   {'was_impossible': False} Low	SZA
user: 9388585797 item: 482988610  r_ui = 1.00   est = 1.00   {'was_impossible': False} Doves In the Wind	SZA|Kendrick Lamar
user: 9388585797 item: 1476338697 r_ui = 1.00   est = 1.00   {'was_impos

## 模型存储

In [39]:
import surprise
surprise.dump.dump('./model/recommendation.model', algo=algo)
# 可以用下面的方式载入
algo = surprise.dump.load('./model/recommendation.model')

## 不同的推荐系统算法评估

In [50]:
# 载入数据
import os
from surprise import Reader, Dataset
# 指定文件路径
file_path = os.path.expanduser('./163_music_suprise_format.txt')
# 指定文件格式
reader = Reader(line_format='user item rating timestamp', sep=',')
# 从文件读取数据
music_data = Dataset.load_from_file(file_path, reader=reader)
# 分成5折
# music_data.split(n_folds=5)

In [51]:
music_data

In [52]:
music_data.raw_ratings[:20]

[('10009377589', '2772113', 1.0, '1300000'),
 ('10009377589', '5100462', 1.0, '1300000'),
 ('10009377589', '28718313', 1.0, '1300000'),
 ('10009377589', '518904648', 1.0, '1300000'),
 ('10009377589', '1786865', 1.0, '1300000'),
 ('10009377589', '1571835', 1.0, '1300000'),
 ('10009377589', '461544117', 1.0, '1300000'),
 ('10009377589', '29133008', 1.0, '1300000'),
 ('10009377589', '479223413', 1.0, '1300000'),
 ('10009377589', '29412405', 1.0, '1300000'),
 ('10009377589', '18513043', 1.0, '1300000'),
 ('10009377589', '431610014', 1.0, '1300000'),
 ('10009377589', '1830419924', 1.0, '1300000'),
 ('10009377589', '17793578', 1.0, '1300000'),
 ('10009377589', '2051548110', 1.0, '1300000'),
 ('10009377589', '1934649', 1.0, '1300000'),
 ('10009377589', '1955978156', 1.0, '1300000'),
 ('10009377589', '1420972635', 1.0, '1300000'),
 ('10009377589', '544199008', 1.0, '1300000'),
 ('10009377589', '1985237093', 1.0, '1300000')]

In [57]:
### 使用NormalPredictor
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
algo = NormalPredictor()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
MAE (testset)     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
Fit time          0.05    0.06    0.07    0.06    0.06    0.06    0.00    
Test time         0.14    0.04    0.04    0.13    0.04    0.08    0.05    


In [59]:
### 使用BaselineOnly
algo = BaselineOnly()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
MAE (testset)     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
Fit time          0.19    0.15    0.18    0.17    0.18    0.17    0.01    
Test time         0.05    0.05    0.04    0.04    0.04    0.04    0.00    


In [61]:
### 使用基础版协同过滤
from surprise import KNNBasic
algo = KNNBasic()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
MAE (testset)     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
Fit time          0.02    0.03    0.02    0.03    0.03    0.02    0.00    
Test time         0.12    0.23    0.11    0.12    0.21    0.16    0.05    


In [62]:
### 使用均值协同过滤
from surprise import KNNWithMeans
algo = KNNWithMeans()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
MAE (testset)     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
Fit time          0.03    0.03    0.05    0.04    0.04    0.04    0.00    
Test time         0.17    0.12    0.13    0.13    0.13    0.13    0.02    


In [63]:
### 使用协同过滤baseline
from surprise import KNNBaseline
algo = KNNBaseline()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
MAE (testset)     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
Fit time          0.21    0.16    0.19    0.18    0.21    0.19    0.02    
Test time         0.14    0.13    0.13    0.13    0.13    0.13    0.00    


In [64]:
### 使用SVD
from surprise import SVD
algo = SVD()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0397  0.0401  0.0399  0.0404  0.0403  0.0401  0.0003  
MAE (testset)     0.0180  0.0183  0.0180  0.0182  0.0182  0.0181  0.0001  
Fit time          0.89    0.84    1.09    0.94    0.94    0.94    0.08    
Test time         0.18    0.06    0.06    0.15    0.06    0.10    0.05    


In [65]:
### 使用SVD++
from surprise import SVDpp
algo = SVDpp()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0176  0.0172  0.0174  0.0176  0.0172  0.0174  0.0002  
MAE (testset)     0.0078  0.0078  0.0077  0.0077  0.0076  0.0077  0.0001  
Fit time          6.72    7.02    20.87   15.01   7.25    11.37   5.68    
Test time         0.85    0.84    2.41    0.91    0.83    1.17    0.62    


In [ ]:
### 使用NMF
from surprise import NMF
algo = NMF()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])
print_perf(perf)

## Song2Vec

In [90]:
#一个东西
import multiprocessing
import gensim
import sys
from random import shuffle

# path = "./data/output/popular/"

def parse_playlist_get_sequence(in_line, playlist_sequence):
    song_sequence = []
    contents = in_line.strip().split("\t")
    # 解析歌单序列
    for song in contents[1:]:
        try:
            song_id, song_name, artist, popularity = song.split("::")
            song_sequence.append(song_id)
        except:
            print("song format error")
            print(song+"\n")
    for i in range(len(song_sequence)):
        shuffle(song_sequence)# 歌单中的歌曲id顺序被打乱
        playlist_sequence.append(song_sequence)


def train_song2vec(in_file, out_file):
    #所有歌单序列
    playlist_sequence = []
    #遍历所有歌单
    for line in open(in_file,encoding='utf-8'):
        parse_playlist_get_sequence(line, playlist_sequence)
    #使用word2vec训练
    cores = multiprocessing.cpu_count()
    print("using all "+str(cores)+" cores")
    print("Training word2vec model...")
    # 训练模型，这里的一个songid相当于一个分词，一个歌单相当于一篇文章
    # print("playlist: ", playlist_sequence)
    model = gensim.models.Word2Vec(sentences=playlist_sequence, min_count=3, window=7, workers=cores)
    print("Saving model...")
    model.save(out_file)

# ./data/popular.playlist内容格式：歌单名称##tags##歌单ID##订阅量 + \t +id:::name:::artists::popularity
song_sequence_file = "./songlist2songs.txt"
model_file = "./model/song2vec.model"
train_song2vec(song_sequence_file, model_file)


song format error
26377622::ネプトリュード(Class::NEPTLUDE=>extends.TX_CLUSTERS/.)::Frontier Works::45

song format error
532333::モノノ怪::サミシ怪~samishige~::高梨康治::40

using all 16 cores
Training word2vec model...
Saving model...


In [91]:
import pickle
# 加载歌曲的id——name+"\t"+artist
song_dic = pickle.load(open("./pro_data/song.pkl","rb"))
# 加载训练模型
model = gensim.models.Word2Vec.load(model_file)

In [95]:
# 获取十个歌曲的id
song_id_list = list(song_dic.keys())[1000:1500:50]

for song_id in song_id_list:
    result_song_list = model.wv.most_similar(positive=song_id)
    print(song_id, song_dic[song_id])
    print("\n相似歌曲 和 相似度 分别为:")
    for song in result_song_list:
        print("\t", song_dic[song[0]], song[1])
    print("\n")

1458626811 Substitute	Dawin

相似歌曲 和 相似度 分别为:
	 I Like It	Cardi B|Bad Bunny|J. Balvin 0.8228569626808167
	 Drop	Connor Price|Zensery 0.8013939261436462
	 Lift Me Up	Tyga 0.7936635613441467
	 Depression	Dax 0.7623148560523987
	 universe (sped up)	Thuy 0.7489740252494812
	 Satin Black	Iamjakehill 0.7486222386360168
	 Boyz	Jesy Nelson|Nicki Minaj 0.7450708150863647
	 Nonsense	Sabrina Carpenter 0.7417552471160889
	 Knockout	Noga Erez 0.7375650405883789
	 Hex	Ark Patrol 0.7365177273750305


1424400870 HEARTBREAK ANNIVERSARY	Giveon

相似歌曲 和 相似度 分别为:
	 Bet On Me (slowed + reverb)	Walk off the Earth|D Smoke 0.7749146223068237
	 Cable Car	江语 0.7382282614707947
	 Round Town	Adrian Daniel|Asiah 0.7273243069648743
	 Someone You Loved	Conor Maynard 0.717329204082489
	 Reason	XO TEAM 0.7012072801589966
	 You	Shoffy 0.6985587477684021
	 Moonlight	Kali Uchis 0.6964356303215027
	 IS IT WORTH IT	Rae Khalil 0.6902110576629639
	 No	MADILYN|Megan Nicole 0.6881411075592041
	 softly // clairo cover	mich 0.6813

In [108]:
# 获取十个歌曲的id

user_inner_id = 42
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
songid_list = []
with open('result.txt', 'w', encoding = 'utf-8') as f:
    for song in items:
        # print(song)
        print(song_id_name_dic[algo.trainset.to_raw_iid(song)], file = f)
        songid_list.append(str(algo.trainset.to_raw_iid(song)))
    print("\n相似歌曲 和 相似度 分别为:", file = f)
    result_song_list = model.wv.most_similar(positive=songid_list, topn=10)
    for song in result_song_list:
        print("\t", song_dic[song[0]], song[1], file = f)